In [12]:
from os import listdir, makedirs, path

import queue
import re
import subprocess
import threading

INPUT_PATH = "0801"
OUTPUT_PATH = "0801s"

INPUT_DIRS = [
  '01-COBERTURA PP OESTE',
  '02-COBERTURA ENTRADA PR',
  '03-ENTRADA PP RAIO X - TERREO',
  '04-PALACIO ALA LESTE - TERREO',
  '05-PALACIO ALA OESTE',
  '06-PRIVATIVO COMBOIO',
  '07-SALA PG - 3 PISO',
  '08-SUBSOLO ADM',
  '09-ELEVADOR DE SERVICO - TERREO',
  '10-SALAO NOBRE',
  '11-SALAO OESTE',
  '12-2º ANDAR RAMPA',
  '13-2º ANDAR ELEVADOR LESTE',
  '14-2º ANDAR ELEVADOR OESTE',
  '15-MESANINO',
  '16-CAMERA-VIP-PR',
  '17 3º ANDAR SALA PR',
  '18-3º ANDAR SALA DE AUDIENCIA',
  '19-3º ANDAR CORREDOR ACESSO LESTE',
  '20-ELEVADORES ALA OESTE',
  '21-2º ANDAR ESCADA OESTE',
  '22-3º ANDAR ESCADA OESTE',
  '23-Elevador Ministros',
  '24-4° Andar - Elevador Leste',
  '25-4° Andar - Escada Oeste',
  '26-4º Andar - Elevador Oeste',
  '27-4º Andar - Lado Leste',
  '28-4º Andar - Lado Oeste',
  '29-Anexo I',
  '30-Anexo I - Entrada Principal Catracas',
  '31-Anexo I - Concha Anexo',
  '32-Anexo III - Refeitorio Creden',
  '33-Anexo III - Refeitorio',
]

FFMPEG_CMD = 'ffmpeg -i "INPUT_PATH" -vf scale=500:-2 -an "OUTPUT_PATH"'

In [ ]:
def clean_name(inp):
  out_name = inp.replace(" ", "-")
  out_name = re.sub(r'[º°]', 'o', out_name)
  out_name = re.sub(r'[-]+', '-', out_name)
  out_name = re.sub(r'[_]+', '_', out_name)
  return out_name.lower()

In [13]:
def run_ffmpeg(q):
  while True:
    try:
      ip, op = q.get(timeout=3)
    except queue.Empty:
      return
    my_cmd = FFMPEG_CMD.replace("INPUT_PATH", ip).replace("OUTPUT_PATH", op)
    print(my_cmd)
    # ret = subprocess.run(my_cmd, shell=True)
    q.task_done()

In [ ]:
to_process = queue.Queue()

for input_dir in INPUT_DIRS:
  output_dir = clean_name(input_dir)
  output_dir_path = path.join(OUTPUT_PATH, output_dir)
  makedirs(output_dir_path, exist_ok=True)

  input_dir_path = path.join(INPUT_PATH, input_dir)
  input_files = [f for f in listdir(input_dir_path) if f.endswith("mp4")]
  for input_file in input_files:
    output_file = clean_name(input_file)
    output_file_path = path.join(output_dir_path, output_file)
    if not path.exists(output_file_path):
      input_file_path = path.join(input_dir_path, input_file)
      to_process.put((input_file_path, output_file_path))

In [ ]:
for _ in range(4):
  threading.Thread(target=run_ffmpeg,
                   args=(to_process)).start()

to_process.join()
print("DONE")